In [1]:
from PIL import Image
from PIL import ImageDraw
import timeit
import glob
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from skimage.feature import greycomatrix, greycoprops
import matplotlib.image as mpimg 
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import pandas as pd
from sklearn import preprocessing
import random
from sklearn.model_selection import train_test_split  
from sklearn.naive_bayes import GaussianNB  
from sklearn.metrics import accuracy_score

In [9]:
map_8bit_to_3bit = [i // 32 for i in range(256)]
model = {}
model_test = {}
neighbor = 3
time_modelling_start = timeit.default_timer()
model["cap"] = []
model["tulis"] = []
model_test["cap"] = []
model_test["tulis"] = []

In [10]:
#Pre Processing
def load_img(img_path):
    return Image.open(img_path).convert('L')

def get_img_size(img):
    return img.size

def print_img(img):
    plt.imshow(img)
    
def get_img_colors(img, sampling_count):
    img_width, img_height = get_img_size(img)
    
    sample_width = int(img_width / sampling_count)
    sample_height = int(img_height / sampling_count)
    
    half_sample_width = int(sample_width/2)
    half_sample_height = int(sample_height/2)

    img_colors = []

    width_constraint = img_width - (2 * sample_width)
    height_constraint = img_height - (2 * sample_height)
   
    row = 0
    for point_x in range(half_sample_width, img_width, sample_width):
        if(row == sampling_count):
            break
        column = 0
        for point_y in range(half_sample_height, img_height, sample_height):
            if(column == sampling_count):
                break
            img_colors.append(img.getpixel((point_x, point_y)))
                
            column += 1
                
        row += 1
            
    return img_colors

def construct_img(img_colors, img_dimension, sampling_count):
    img_new = Image.new('RGB', (img_dimension), (255, 255, 255))
    draw = ImageDraw.Draw(img_new)
    
    img_new_width = img_dimension[0]
    img_new_height = img_dimension[1]

    sample_width = int(img_new_width/sampling_count)
    sample_height = int(img_new_height/sampling_count)
    loop_count = 0

    img_colors_len = len(img_colors)

    for w in range(0, img_new_width, sample_width):
        for h in range(0, img_new_height, sample_height):
            if(loop_count == img_colors_len):
                break
            
            current_color = (img_colors[loop_count], img_colors[loop_count], img_colors[loop_count])
            draw.rectangle((w, h, w+sample_width, h+sample_height), fill=current_color)
            loop_count += 1
    
    return img_new

def get_3bit_colors(img_colors):
    loop_count = 0
    
    for img_color in img_colors:
        img_colors[loop_count] = map_8bit_to_3bit[img_color]
        
        loop_count += 1
        
    return img_colors

def get_img_matrix(img_colors):
    img_matrix = [] 
    loop_count = 0
    
    loop_count = 0
    img_square_dimension = int(math.sqrt(len(img_colors)))
    
    for row in range(img_square_dimension):
        temp_row = []
        for col in range(img_square_dimension):
            temp_row.append(img_colors[loop_count])
            
            loop_count += 1
        img_matrix.append(temp_row)
        
    return img_matrix


In [11]:
#Features Extraction
def feature_extract(img, sampling_count, class_name):
    glcm_component=[]
    img_colors = get_img_colors(img, sampling_count)
    img_3bit_colors = get_3bit_colors(img_colors)
    img_matrix = get_img_matrix(img_3bit_colors)
    glcm_matrix=greycomatrix(img_matrix, distances=[1], angles=[0], levels=12, symmetric=False, normed=False)
    component = ['contrast', 'homogeneity', 'energy', 'correlation', 'ASM', 'dissimilarity']
    for x in component:
        glcm_component.append(greycoprops(glcm_matrix, x)[0][0])
    if(class_name == "tulis"):
        glcm_component.append(1)
    else:
        glcm_component.append(0)
    return(glcm_component)

In [12]:
#Modelling
def get_class_names(training_folder_path):
    return os.listdir(training_folder_path)

def make_a_model(class_name, features, learning_rate):
    random_splitter = random.uniform(0, 1)
    if(random_splitter <= learning_rate):
        model[class_name].append(features)
    else:
        model_test[class_name].append(features)
        
def to_dataframe(model):
    models = pd.DataFrame(model['cap'] + model['tulis'], columns=['contrast', 'homogeneity', 'energy', 'correlation', 'ASM', 'dissimilarity', 'class'])
    return models

In [13]:
#Classification
def naive_bayes(image):
    prob = 1
    probability = 0.00
    for class_name in class_names:
        for i in range(5):
            a = (image[i]-mean[class_name][i])**2/(2*variance[class_name][i])
            b = 1/(np.sqrt(2*3.14*variance[class_name][i]))
            
            prob=prob * (b* np.exp(0-a))
        
        prob=1
        if(prob>probability):
            probability = prob
            probability_class = class_name
    return probability_class

In [14]:
#Validation
def validate(sampling_count):
    right = 0
    total = 0
    for class_name in class_names:
        validate_img_paths = glob.glob(validation_folder_path + class_name + '/' +img_type)
        for images in validate_img_paths:
            img = load_img(images)
            training_img_class_name = class_name
            image = feature_extract(img, sampling_count, training_img_class_name)
            predict = naive_bayes(image)
            class_image = class_name
            if(predict == class_name):
                   right+=1
            total +=1
        accuracy = (right/total) * 100
    print("Accuracy: " + str(accuracy) +"%")

In [15]:
#Training Testing

learning_rate = 1
sampling_count = 128
img_type = '*.jpg'
root_path = './'
training_folder_path = root_path + 'glcm/training/'
validation_folder_path = root_path + 'glcm/validation/'
test_folder_path = root_path + 'glcm/test/'

class_names = get_class_names(training_folder_path)

for i in range(1):
    for class_name in class_names:
        training_img_paths = glob.glob(training_folder_path + class_name + '/' + img_type)
        for training_img_path in training_img_paths:
            training_img = load_img(training_img_path)
            training_img_class_name = class_name
            feature = feature_extract(training_img, sampling_count, training_img_class_name)
            make_a_model(class_name, feature, learning_rate)
        
    models = to_dataframe(model)
    models_test = to_dataframe(model_test)

#     testings()
# validate(128)



time_modelling_stop = timeit.default_timer()
print('Time elapsed to make this model is ' + str(time_modelling_stop - time_modelling_start) + ' seconds.')

Time elapsed to make this model is 15.413915199999998 seconds.


In [27]:
x = models.iloc[:, [0, 5]].values  
y = models.iloc[:, 6].values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 5)
sc = StandardScaler()  
x_train = sc.fit_transform(x_train)  
x_test = sc.transform(x_test)
classifier = GaussianNB()  
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
accuracy_score(y_test, y_pred) 

0.7014925373134329

In [31]:
x_test

array([[-1.11283848e+00, -1.41258054e+00],
       [-3.03596402e-01, -2.48111692e-01],
       [-1.89323854e-01,  1.51658468e-02],
       [-4.58618894e-01, -2.21071591e-01],
       [-1.07015590e+00, -1.17445320e+00],
       [-5.85638649e-01, -6.97180890e-01],
       [-1.71436644e-01, -1.55942747e-01],
       [ 2.87152740e+00,  2.63703797e+00],
       [-6.59531333e-01, -5.85386065e-01],
       [ 2.03021170e+00,  1.80053937e+00],
       [ 5.47382501e-01,  6.62674493e-01],
       [-2.94755597e-01, -1.32537068e-01],
       [-7.16383449e-02,  2.25090069e-01],
       [-6.66192777e-01, -6.70867674e-01],
       [-9.99429454e-01, -9.84590992e-01],
       [-3.37684902e-01, -1.37479882e-01],
       [ 2.26175856e+00,  2.18564274e+00],
       [-8.68667776e-01, -8.37760338e-01],
       [-3.70210841e-01, -4.25180738e-01],
       [-3.73853646e-02,  6.56116259e-02],
       [ 7.69694230e-02,  2.87311376e-01],
       [ 6.41999677e-01,  7.77958363e-01],
       [-4.20171671e-01, -5.54711543e-01],
       [ 1.

In [7]:
for j in range(2,20):
    for i in range(2,j):
        print(i,j)

2 3
2 4
3 4
2 5
3 5
4 5
2 6
3 6
4 6
5 6
2 7
3 7
4 7
5 7
6 7
2 8
3 8
4 8
5 8
6 8
7 8
2 9
3 9
4 9
5 9
6 9
7 9
8 9
2 10
3 10
4 10
5 10
6 10
7 10
8 10
9 10
2 11
3 11
4 11
5 11
6 11
7 11
8 11
9 11
10 11
2 12
3 12
4 12
5 12
6 12
7 12
8 12
9 12
10 12
11 12
2 13
3 13
4 13
5 13
6 13
7 13
8 13
9 13
10 13
11 13
12 13
2 14
3 14
4 14
5 14
6 14
7 14
8 14
9 14
10 14
11 14
12 14
13 14
2 15
3 15
4 15
5 15
6 15
7 15
8 15
9 15
10 15
11 15
12 15
13 15
14 15
2 16
3 16
4 16
5 16
6 16
7 16
8 16
9 16
10 16
11 16
12 16
13 16
14 16
15 16
2 17
3 17
4 17
5 17
6 17
7 17
8 17
9 17
10 17
11 17
12 17
13 17
14 17
15 17
16 17
2 18
3 18
4 18
5 18
6 18
7 18
8 18
9 18
10 18
11 18
12 18
13 18
14 18
15 18
16 18
17 18
2 19
3 19
4 19
5 19
6 19
7 19
8 19
9 19
10 19
11 19
12 19
13 19
14 19
15 19
16 19
17 19
18 19
